# import library

In [83]:
import pandas as pd
pd.set_option('display.max_columns', 100)
import numpy as np

from bs4 import BeautifulSoup
import requests
import json

import re
from pprint import pprint
from tqdm import tqdm_notebook as tqdm

from keys import *

# scrape from adult school directory

In [31]:
def get_content_with_keyword(keyword, name):
    target = [pp for pp in as_soup.find_all("p") if keyword in pp.get_text()]
    
    if keyword == "Street Address":
        target = target[0].find("span").get_text().strip(name).strip()
        
    else:
        target = target[0].find("span").get_text()

    return target

In [32]:
with open("../data/adult_school/as_code.html") as f:
    ass_html = f.read()
ass_soup = BeautifulSoup(ass_html)

# get all td tags
td_header2 = ass_soup.find_all("td", headers="header2")

ass_dict = {"name": [], "url": [], "Agency Type": [],
            "Street Address": []}

# extracet and store href and text from each td tag
for td in td_header2:
    aa = td.find("a")
    url = aa.get("href")
    name = aa.get_text()
    ass_dict["name"].append(name)
    ass_dict["url"].append(url)
    
# extract agency type, address
for url, name in zip(tqdm(offices_dict["url"]), offices_dict["name"]):
    
    as_html = requests.get(url).text
    as_soup = BeautifulSoup(as_html)
    
    keywords = ["Agency Type", "Street Address"]
    
    for kw in keywords:
        target = get_content_with_keyword(kw, name)
        ass_dict[kw].append(target)
        
ass_df = pd.DataFrame(ass_dict)
ass_df.columns = ass_df.columns.str.replace(" ", "_").str.lower()

print(ass_df.shape)
ass_df.head()

(706, 4)


,name,url,agency_type,street_address
0,Department of Developmental Services,https://www.otan.us/caaeproviders/index.cfm?fu...,State Agency,"1600 9th Street, #340, MS 3-17 Sacramento, CA ..."
1,Office of Correctional Education,https://www.otan.us/caaeproviders/index.cfm?fu...,State Agency,"1515 S Street Room 400-South Sacramento, CA 95811"
2,Sacramento City Adult Education Office,https://www.otan.us/caaeproviders/index.cfm?fu...,Local Educational Agency (LEA),"5241 J Street Sacramento, CA 95819"
3,Washington Adult School,https://www.otan.us/caaeproviders/index.cfm?fu...,Local Educational Agency (LEA),"919 Westacre Road West Sacramento, CA 95691"
4,Natomas Adult Education Program - Discover HS,https://www.otan.us/caaeproviders/index.cfm?fu...,Local Educational Agency (LEA),"3401 Rosin Boulevard Sacramento, CA 95834"


# geocoding

In [84]:
def try_mapbox_request(col_val):
    endpoint = 'https://api.mapbox.com/geocoding/v5/mapbox.places/'
    
    params = {'limit': 1,
              'access_token': mapbox_access_token}

    url = requests.Request('GET', endpoint+col_val+'.json', params=params).prepare().url
    response = requests.get(url)
    results = response.text
    
    return results

def geocoder(df, cols_to_search):

    latitude = []
    longitude = []

    for col_val0, col_val1 in zip(tqdm(df[cols_to_search[0]]), df[cols_to_search[1]]):

        results = try_mapbox_request(col_val0)
        
        if "Not Found" in results:
            results = try_mapbox_request(col_val1 + " California")
    
        data = json.loads(results)
        
        print("\n" + col_val0)
        pprint(data)
        
        if len(data["features"]) == 0:
            longitude.append(np.nan)
            latitude.append(np.nan)
            
        else:
            longitude.append(data["features"][0]["center"][0])
            latitude.append(data["features"][0]["center"][1])

    df["longitude"] = longitude
    df["latitude"] = latitude
    
    return df

In [80]:
endpoint = 'https://api.mapbox.com/geocoding/v5/mapbox.places/'

params = {'limit': 1,
          'access_token': mapbox_access_token}

col_val = ass_df["street_address"][120]
print(col_val)

url = requests.Request('GET', endpoint+col_val+'.json', params=params).prepare().url
response = requests.get(url)
results = response.text
data = json.loads(results)
len(data["features"]) == 0

, 94607


True

In [85]:
cols_to_search = ["street_address", "name"]

ass_df = geocoder(ass_df, cols_to_search)


1600 9th Street, #340, MS 3-17 Sacramento, CA 95814
{'attribution': 'NOTICE: © 2019 Mapbox and its suppliers. All rights reserved. '
                'Use of this data is subject to the Mapbox Terms of Service '
                '(https://www.mapbox.com/about/maps/). This response and the '
                'information it contains may not be retained. POI(s) provided '
                'by Foursquare.',
 'features': [{'center': [-121.49764, 38.57301],
               'context': [{'id': 'neighborhood.293431', 'text': 'Downtown'},
                           {'id': 'postcode.12986816655967210',
                            'text': '95814'},
                           {'id': 'place.4559583397685100',
                            'text': 'Sacramento',
                            'wikidata': 'Q18013'},
                           {'id': 'region.11319063928738010',
                            'short_code': 'US-CA',
                            'text': 'California',
                            'wikid

,name,url,agency_type,street_address,longitude,latitude
0,Department of Developmental Services,https://www.otan.us/caaeproviders/index.cfm?fu...,State Agency,"1600 9th Street, #340, MS 3-17 Sacramento, CA ...",-121.497640,38.573010
1,Office of Correctional Education,https://www.otan.us/caaeproviders/index.cfm?fu...,State Agency,"1515 S Street Room 400-South Sacramento, CA 95811",-121.490177,38.568732
2,Sacramento City Adult Education Office,https://www.otan.us/caaeproviders/index.cfm?fu...,Local Educational Agency (LEA),"5241 J Street Sacramento, CA 95819",-121.439157,38.566467
3,Washington Adult School,https://www.otan.us/caaeproviders/index.cfm?fu...,Local Educational Agency (LEA),"919 Westacre Road West Sacramento, CA 95691",-121.529277,38.577762
4,Natomas Adult Education Program - Discover HS,https://www.otan.us/caaeproviders/index.cfm?fu...,Local Educational Agency (LEA),"3401 Rosin Boulevard Sacramento, CA 95834",-121.502870,38.630750


In [87]:
print(ass_df.shape)
ass_df.head()

(706, 6)


,name,url,agency_type,street_address,longitude,latitude
0,Department of Developmental Services,https://www.otan.us/caaeproviders/index.cfm?fu...,State Agency,"1600 9th Street, #340, MS 3-17 Sacramento, CA ...",-121.497640,38.573010
1,Office of Correctional Education,https://www.otan.us/caaeproviders/index.cfm?fu...,State Agency,"1515 S Street Room 400-South Sacramento, CA 95811",-121.490177,38.568732
2,Sacramento City Adult Education Office,https://www.otan.us/caaeproviders/index.cfm?fu...,Local Educational Agency (LEA),"5241 J Street Sacramento, CA 95819",-121.439157,38.566467
3,Washington Adult School,https://www.otan.us/caaeproviders/index.cfm?fu...,Local Educational Agency (LEA),"919 Westacre Road West Sacramento, CA 95691",-121.529277,38.577762
4,Natomas Adult Education Program - Discover HS,https://www.otan.us/caaeproviders/index.cfm?fu...,Local Educational Agency (LEA),"3401 Rosin Boulevard Sacramento, CA 95834",-121.502870,38.630750


In [96]:
ass_df.iloc[120]

name                                                  Laney College
url               https://www.otan.us/caaeproviders/index.cfm?fu...
agency_type                                                        
street_address                                              , 94607
longitude                                                  -123.215
latitude                                                     39.161
Name: 120, dtype: object

In [94]:
ass_df[ass_df["longitude"].isnull()]

,name,url,agency_type,street_address,longitude,latitude
246,Ukiah Adult School,https://www.otan.us/caaeproviders/index.cfm?fu...,,", 95482",NaN,NaN
426,YPI FamilySource Center,https://www.otan.us/caaeproviders/index.cfm?fu...,Community-based Organization,"1075 N. Western Ave. #110 Los Angeles, CA 91331",NaN,NaN
627,Crafton Hills College,https://www.otan.us/caaeproviders/index.cfm?fu...,,", 92399",NaN,NaN


# manual correction

In [97]:
ass_df.iloc[120, 4] = -122.262895
ass_df.iloc[120, 5] = 37.796879

ass_df.iloc[246, 4] = -123.215087
ass_df.iloc[246, 5] = 39.161013

ass_df.iloc[426, 4] = -118.309713
ass_df.iloc[426, 5] = 34.090462

ass_df.iloc[627, 4] = -117.100611
ass_df.iloc[627, 5] = 34.038853

In [105]:
ass_df[ass_df["longitude"] > -90]

,name,url,agency_type,street_address,longitude,latitude
227,California Community Opportunities,https://www.otan.us/caaeproviders/index.cfm?fu...,,"3315 Almaden Expressway, Suite #20 San Jose, C...",-85.780396,38.23946


In [106]:
ass_df.iloc[227, 4] = -121.879331
ass_df.iloc[227, 5] = 37.273769

In [107]:
ass_df.to_csv("../data/resources/adult_school_locations.csv")